## <span style="color:blue">PL/SQL </span> 

In this section, we introduce PL/pgSQL on postgreSQL. For Oracle developpers, the syntax is very similar.

Before going on, make sure to etablish a connexion :

In [1]:
-- connection: host='localhost' dbname='ds2' user='ds2' 


 ## <span style="color:blue">Let's start </span>

A PL/SQL program consists of 4 sections :

DECLARE (optional)
Variable definition 

BEGIN (mandatory)
Implements the business logic and needs to contain at least one declaration SQL or PL/SQL

EXCEPTION (optional)
Exception management 

END; ( mandatory ) 







## <span style="color:blue">Enabling Output of a PL/SQL Block</span> 

To enable output in postgreSQL, execute the following command before running the PL/SQL block


In [2]:
set client_min_messages = LOG;

You can filter the trace level to DEBUG5, DEBUG4, DEBUG3, DEBUG2, DEBUG1, LOG, NOTICE, WARNING, ERROR, FATAL, and PANIC.

Ready for your first procedure :

In [ ]:
DO $$
BEGIN
raise log 'Hello';
END;
$$

## <span style="color:blue">Your first Program</span>

PL/SQL is a language that may be used in some specific database objects : 
* Triggers
* Stored procedures
* Functions
* Anonymous block

In PL/SQL, you can run a PL/SQL code without creating a database object like a stored procedure or a trigger, it is what we call an Anonymous block. In postgres, to run an anonymous block you can do it with the DO command :


In [ ]:

DO $$                      -- DO (mandatory) indicates to Postgres it's an anonymous block.
<< label >>                -- you can define a label we will see the interest  later
DECLARE                    -- Optional section for declaring variables
BEGIN                     -- computing block (mandatory) : have to contain at least one instruction
   Raise notice 'Hello';   -- Print Hello
END                        -- computing block end (mandatory)
$$    

Note that the double dollar ($$) is a substitution of a single quote (‘).

Bravo, you ran your first PL/SQL program.

Here, we create the same program with via a function :

In [4]:
DROP function if exists f_hello( text);

CREATE or replace FUNCTION f_hello(v_myTxt text) RETURNS text 
AS $BODY$
DECLARE
  v_hello text ='Hello';
BEGIN
  RETURN v_hello||' '||v_myTxt;
END
$BODY$
LANGUAGE plpgsql; 


Now, we can call our function :

In [ ]:
select f_hello('Guy');

We can do the same thing through a stored procedure :

In [3]:
DROP procedure if exists p_hello( varchar(50));

CREATE PROCEDURE p_hello(v_myTxt varchar(50)) 
LANGUAGE plpgsql 
AS $BODY$ 
DECLARE
   v_hello text ='Hello';
BEGIN
   Raise notice '% : % ', v_hello, v_myTxt;
END
$BODY$;

In [ ]:
call p_hello('Guy');


# <span style="color:blue">Scalar variables</span> 

If you are famillar with programming it's very similar, you declare the name, the datatype and evantually the default value.

The datatype list supported in postgres is available here :
https://www.postgresql.org/docs/11/datatype.html

Here is a simple code displaying a variable :

In [ ]:
DO $$                      
DECLARE
costype varchar(50) default 'Galaxy';
BEGIN                     
   Raise info 'var costype : %',costype; 
END  
$$ 
LANGUAGE plpgsql;

You can define the type of a variable from : 
* an existing colum  with the syntax %TYPE
* an existing local variable 

In [ ]:
DO $$                      
DECLARE
v_myTitle products.title%TYPE;
v_copyMyTitleType v_myTitle%TYPE;
BEGIN  
   v_myTitle := 'Coucou';
   Raise info 'v_myTitle : %',v_myTitle; 
   v_copyMyTitleType := length(v_myTitle)::text; -- we need to cast data from int to text
   Raise info 'v_copyMyIntType : %',v_copyMyTitleType; 
END  
$$ 
LANGUAGE plpgsql;

# <span style="color:blue">Composite variables</span> 

Now, let's play with composite variables.<br/>
When you need to collect a row with a set of attribute you can do it as follows :

In [ ]:
DO $$                      
DECLARE
rt_myrow categories%ROWTYPE;  
rec_myrow RECORD;  
BEGIN                     
   SELECT * into strict rt_myrow FROM categories order by category LIMIT 1 OFFSET 5 ; --STRICT raise an error if multi rows
   Raise info 'Variable with ROW TYPE : Id category : % - Category Name : % ',rt_myrow.category,rt_myrow.categoryname;                                                                                                               
   SELECT * into strict rec_myrow FROM categories order by category LIMIT 1 OFFSET 7 ; --STRICT raise an error if multi rows
   Raise info 'Variable with RECORD : Id category : % - Category Name : % ',rec_myrow.category,rec_myrow.categoryname;                                                                                                               
END                        
$$


In [ ]:
DO $$                      
DECLARE  
rec_myrow RECORD;
BEGIN                     
   SELECT * into strict rec_myrow FROM categories order by category LIMIT 1 OFFSET 5 ; --STRICT raise an error if multi rows
   Raise info 'Variable with ROW TYPE : Id category : % - Category Name : % ',rec_myrow.category,rec_myrow.categoryname;                                                                                                               
   SELECT * into strict rec_myrow FROM products order by prod_id LIMIT 1 OFFSET 7 ; --STRICT raise an error if multi rows
   Raise info 'Variable with RECORD : Id product : % - Product Name : % ',rec_myrow.prod_id,rec_myrow.title;        
END                        
$$

Are you ok with variables ?<br/>
From the preceding example, could you write an anonymous block to display the 11th row of the category table and get back the result into a composite variable with your own type ?<br/>


Now you are ready to ready to developp in PL/SQL ;-).

 ### <span style="color:red">EXERCISES </span>

 ### <span style="color:blue">EX - 1 </span>
 
 Create an anonymous block computing the hypotenuse of a right-angled triangle where the opposite side measures 5.5 and the adjacent side measures 10.<br/>

All mathematique operators are <a href="https://docs.postgresql.fr/11/functions-math.html">here</a>

 ### <span style="color:blue">EX - 2 </span>
 
 Transform the anonymous block to a function "hypo" accepting 2 arguments a and b.

In [ ]:
select f_hypo(3.5,56.4);

 ### <span style="color:blue">EX - 3 </span>

Guess the output of this anonymous procedure :

In [ ]:
DO $$                      
<< level1 >>
DECLARE
v_level otypedef.otype_descr%TYPE = 'level1';
BEGIN 
   Raise info 'bloc1 -> level = %' ,v_level;  
   << level2_1 >>
   DECLARE
   v_level otypedef.otype_descr%TYPE = 'level2_1';
   BEGIN
       Raise info 'bloc2_1 -> level = % ',v_level; 
       Raise info 'bloc2_1 -> level1.level = % ',level1.v_level; 
       << level3_1 >>
       DECLARE
       v_level otypedef.otype_descr%TYPE = 'level3_1_1';
       BEGIN    
           Raise info 'bloc3_1_1 -> level = % ',v_level; 
           Raise info 'bloc3_1_1 -> level1.level = % ',level1.v_level;     
           Raise info 'bloc3_1_1 -> level2_1.level = % ',level2_1.v_level;  
           raise exception 'test';
        END level3_1;
        EXCEPTION
            WHEN others then
            Raise info 'Erreur id  % ',SQLSTATE; 
            Raise info 'Erreur detected  % ',SQLERRM; 
   END level2_1;
   << level2_2 >>
   DECLARE
   v_level otypedef.otype_descr%TYPE = 'level2_2';
   BEGIN
       Raise info 'bloc2_2 -> level = % ',v_level; 
       Raise info 'bloc2_2 -> level1.level = % ',level1.v_level; 
   END level2_2;
END level1 ;                     
$$ 


  ### <span style="color:blue">EX - 4</span>

In this exercice, you will create a course table as defined below :

In [ ]:
drop table if exists courses;
CREATE TABLE  courses
(
  idcourse character varying(10),
  room character varying(100),
  teacher character varying(50),
  teacher_phone character varying(10)
);
insert into courses values ('CS101','Hall 20','George','0651482192');
insert into courses values ('CS154','Auditorium 01','Atkins','0651927291');
insert into courses values ('CS152','Hall 21','Atkins','0651927291');
insert into courses values ('CS102','Hall 21','George','0651482192');

This relation is not very well designed, could you transform it in the Boyce-Codd normal form ?<br/>
Define and create all relations that will be used to store data in the new data model :

Now you are ready to migrate courses to your new design.

Create a stored procedure that will handle data from the courses table and will translate to your new design schema.


In [ ]:
call courseMigration();

Check all your data has migrated well

 ### <span style="color:blue">EX - 5 </span>

Guess the output of this anonymous block :

In [ ]:
DO $$
DECLARE
    cur_ref refcursor;
    rec_row RECORD;
BEGIN
    OPEN cur_ref FOR SELECT * FROM categories  order by category;
    FETCH FIRST FROM cur_ref into rec_row;
    IF NOT FOUND THEN
        raise info 'No data found';
        raise exception 'No data found';
    END IF;
    FETCH cur_ref into rec_row;
    raise info 'Cursor position %', rec_row.category;
    MOVE NEXT FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    MOVE FORWARD 2 FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    raise info 'Cursor position %', rec_row.category;
    
    MOVE LAST FROM cur_ref;
    FETCH cur_ref INTO rec_row;
    IF NOT FOUND THEN
        raise info 'Cursor out of result';
        MOVE RELATIVE -2 from cur_ref;
        FETCH cur_ref INTO rec_row;
    END IF;
    raise info 'Cursor position%', rec_row.category;
    CLOSE cur_ref;
END;
$$ 
;


### <span style="color:blue">EX - 6 </span>
Create a function f_getWay accepting a customerid and returning street name of a customer.<br/>
Use the address1 column from the customers table.<br/>
You may use the string function split_part to help you.<br/>

All string functions are described <a href="https://www.postgresql.org/docs/current/functions-string.html">here</a>






In [ ]:
DROP function if exists f_getWay(integer);


In [ ]:
select f_getWay(1);

 ### <span style="color:blue">EX - 7 </span>
 Write a function "f_hi" taking one argument "username" and return either "Good morning" or "Good afternoon"   according to the current time (now).<br/>
Help : you could use the function now() to get back the current time and to_char() to get the hour.
 


In [ ]:
select f_hi('Hugo');

### <span style="color:blue">EX - 8 </span>


Create a function getExpensiveProduct that Reads the whole products table with a cursor and <br/>
displays with 'raise info' all products names more expensive than the input argument v_maxprice.
The function will return the number of products.

In [ ]:
select getExpensiveProduct(29.9);

### <span style="color:blue">EX - 9 </span>
Write a function "f_inverse" that inverses the input value and returns the result.<br/>
Help  : use the WHILE control structure


In [ ]:
select f_inverse('toto');

### <span style="color:blue">EX - 10 </span>

Write an anonymous block which counts in backwards from 10 to 0 by step 2

Write a function which deletes all rows from the test table lower than i and return the number of deleted rows :<br/>
Help : try to do it with a parameterized cursor and delete rows from the cursor.

Run your function with i = 5

Check your data is well cleaned up.

In [ ]:
select f_truncate('test');

### <span style="color:blue">EX - 11</span>

Modify the following program in order to catch the exception and raise a warning message describing the error encountered.

In [ ]:
DO $$                      
DECLARE
myrow_rowtype categories%ROWTYPE;                                                         
BEGIN                     
   SELECT * FROM categories into strict myrow_rowtype ;
   Raise info 'Id category : % - Category Name : % ',myrow_rowtype.category,myrow_rowtype.categoryname; 
-- Handle exceptions
END                        
$$

Write a division function where inputs are 2 floats and return the  result of the division


In [ ]:
select division(6,7)

What happens if you set up the second argument to 0

In [ ]:
select division(6,0)

Catch the exception and return the value Nan :

In [ ]:
select division(6,0)

### <span style="color:blue">EX - 12 </span>

In this exercice, we will trace all modifications (delete, insert, update ) run on the products tables into an audit table products_audit.
In the products_audit table, we will save all old values in product before modification.
Help : you could use the following DDL to create the products_audit table.


In [ ]:
CREATE TABLE products_audit
(
    operation         char(1)   NOT NULL,
    stamp             timestamp NOT NULL,
    userid            text      NOT NULL,
    prod_id serial NOT NULL,
    category smallint NOT NULL,
    title text NOT NULL,
    actor text NOT NULL,
    price numeric NOT NULL,
    special smallint,
    common_prod_id integer NOT NULL
)

Create the "f_products_audit" function that will save all old values 

Create a trigger "tr_products_audit" firing the  f_products_audit before commiting modifications.

Try to update/ insert a new product :

In [ ]:
select title,price from products where title ='ACADEMY BEAR';

In [ ]:

update products set price= price * 1.2 where title ='ACADEMY BEAR';
select title,price from products where title ='ACADEMY BEAR';

Check your products_audit table :

In [ ]:
select now(),* from products_audit;

### <span style="color:blue">EX - 13 </span>

The manager wants to see all orders per customer.<br/>
For this exercice, we will limit orders to the customer where customerid is 11769.<br/>
Help: you will need to access to the customers and orders table.

Create a view 'view_custOrders' displaying all orders attached to the customerid 11769 ?<br/>
Note that the view should contain following informations : customerid, firstname, lastname, orderdate, tax, orderid 


Check the content of the view 

In [ ]:
select * from view_custOrders;

From the view_custOrders view, the manager must be able to delete and update the customer orders.

Create a function update_custOrder_view that will allow  :
 * to delete any order
 * to update following fields : firstname, lastname, orderdate, tax


Create the trigger that will fire the update_custOrder_view function.

Delete the order (21499)

In [ ]:
delete from view_custOrders where orderid=21499;

Check the order (21499) is no more available 

In [ ]:
select * from view_custOrders;

Check the order is deleted from ordelines and orders tables :

In [ ]:
select * from orderlines where orderid=21499;

In [ ]:
select * from orders where orderid=21499;

Now increase the tax to 20 for the order (37795)

In [ ]:
update view_custOrders set tax=20 where orderid=25278;

Check the view and tables are updated correctly

In [ ]:
select * from view_custOrders where orderid=25278;

In [ ]:
select * from orders where orderid=25278;

### <span style="color:blue">EX - 14 </span>

Write a py_max function returning the maximum value between 2 arguments

In [ ]:
select pymax(2,4)